<a href="https://colab.research.google.com/github/DharmeshRV/Bike-Sharing-Demand-Prediction/blob/main/Bike_Sharing_Demand_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Seoul Bike Sharing Demand Prediction </u></b>

##### **Project Type**    - Regression
##### **Contribution**    - Individual

# **Problem Statement**


**Seoul Bike has become the capital city's popular public transport system. The public bicycle rental system is favored by Seoulites who wish to travel short distances of a few kilometers instead of using crowded buses or subway trains.**

**Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bikes available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.**

# **GitHub Link -**

# ***Let's Begin !***

## ***1. Knowing the Data***

### Import Libraries

In [3]:
# First we import the important  libraries 
import numpy as np
from numpy import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### Dataset Loading

In [4]:
# Now mount the drive to get the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Get the path of the data file
path='/content/drive/MyDrive/Colab Notebooks/Almabetter_Data Science/Capstone Projects/Bike Sharing Demand Prediction - Dharmesh Kumar/SeoulBikeData.csv'

In [6]:
# Now we create the dataframe 
bike_df=pd.read_csv(path,encoding = 'cp1252')

### Dataset First View

In [7]:
bike_df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [8]:
# Dataset Rows and Columns Count
bike_df.shape

(8760, 14)

### Dataset Information

In [9]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

### Duplicate Values

In [10]:
# looking for duplicate entries. True shows duplicate row.
bike_df.duplicated().value_counts()

False    8760
dtype: int64

### Null Values

In [11]:
# getting column-wise null values
bike_df.isnull().sum().sort_values(ascending=False)

Date                         0
Rented Bike Count            0
Hour                         0
Temperature(°C)              0
Humidity(%)                  0
Wind speed (m/s)             0
Visibility (10m)             0
Dew point temperature(°C)    0
Solar Radiation (MJ/m2)      0
Rainfall(mm)                 0
Snowfall (cm)                0
Seasons                      0
Holiday                      0
Functioning Day              0
dtype: int64

### What did we know about our dataset

The dataset has information of rented bikes on hourly basis for one year. We have to analyze the rented bike count per hour and various factors affecting the rentals. The dataset has 8760 rows & 14 columns with no duplicate and null values.

## ***2. Understanding The Variables***

In [12]:
# columns in the dataframe
bike_df.columns

Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day'],
      dtype='object')

In [13]:
# showing some statistical measures 
bike_df.describe(include='all')

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
count,8760,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760,8760,8760
unique,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,2
top,01/12/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring,No Holiday,Yes
freq,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2208,8328,8465
mean,NaN,704.602055,11.500000,12.882922,58.226256,1.724909,1436.825799,4.073813,0.569111,0.148687,0.075068,NaN,NaN,NaN
std,NaN,644.997468,6.922582,11.944825,20.362413,1.036300,608.298712,13.060369,0.868746,1.128193,0.436746,NaN,NaN,NaN
min,NaN,0.000000,0.000000,-17.800000,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,191.000000,5.750000,3.500000,42.000000,0.900000,940.000000,-4.700000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,504.500000,11.500000,13.700000,57.000000,1.500000,1698.000000,5.100000,0.010000,0.000000,0.000000,NaN,NaN,NaN
75%,NaN,1065.250000,17.250000,22.500000,74.000000,2.300000,2000.000000,14.800000,0.930000,0.000000,0.000000,NaN,NaN,NaN


### Checking unique values in each column

In [14]:
# Check Unique Values for each variable.
for i in bike_df.columns.tolist():
  print("No. of unique values in ",i,"is",bike_df[i].nunique(),".")

No. of unique values in  Date is 365 .
No. of unique values in  Rented Bike Count is 2166 .
No. of unique values in  Hour is 24 .
No. of unique values in  Temperature(°C) is 546 .
No. of unique values in  Humidity(%) is 90 .
No. of unique values in  Wind speed (m/s) is 65 .
No. of unique values in  Visibility (10m) is 1789 .
No. of unique values in  Dew point temperature(°C) is 556 .
No. of unique values in  Solar Radiation (MJ/m2) is 345 .
No. of unique values in  Rainfall(mm) is 61 .
No. of unique values in  Snowfall (cm) is 51 .
No. of unique values in  Seasons is 4 .
No. of unique values in  Holiday is 2 .
No. of unique values in  Functioning Day is 2 .


In [15]:
# Unique values in 'Seasons','Holiday', 'Functioning Day' columns
j=['Seasons','Holiday', 'Functioning Day']
for i in j:
  print("Unique values in ",i,"are",bike_df[i].unique(),".")

Unique values in  Seasons are ['Winter' 'Spring' 'Summer' 'Autumn'] .
Unique values in  Holiday are ['No Holiday' 'Holiday'] .
Unique values in  Functioning Day are ['Yes' 'No'] .


### <b> Data Description </b>

### <b> The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.</b>


### <b>Attribute Information: </b>

* ### Date : day-month-year
* ### Rented Bike count - Count of bikes rented at each hour
* ### Hour - Hour of the day
* ### Temperature-Temperature in Celsius
* ### Humidity - %
* ### Windspeed - m/s
* ### Visibility - 10m
* ### Dew point temperature - Celsius
* ### Solar radiation - MJ/m2
* ### Rainfall - mm
* ### Snowfall - cm
* ### Seasons - Winter, Spring, Summer, Autumn
* ### Holiday - Holiday/No Holiday
* ### Functional Day - No(Non Functional Hours), Yes(Functional hours)

## 3. ***Data Wrangling***

First of all we rename the columns to understandable names that are easy to use.

In [16]:
# Code to rename the columns
bike_df.rename(columns = {'Date':'date','Rented Bike Count':'rented_count', 'Hour':'hour','Temperature(°C)':'temp', 'Humidity(%)':'humidity',
                          'Wind speed (m/s)':'wind_speed','Visibility (10m)':'visibility','Dew point temperature(°C)':'dew_point_temp',
                          'Solar Radiation (MJ/m2)':'solar_radiation','Rainfall(mm)':'rainfall','Snowfall (cm)':'snowfall','Seasons':'seasons',
                          'Holiday':'holiday','Functioning Day':'funct_day'},inplace=True)

We have 'date' column with object data type so first we change it to datetime datatype. Further we create a column for days using the 'date' column because days of the week may exhibit some kind of renting pattern.



The dataset has a 'season' column with 4 seasons of the year so we are not considering to analyze the month-wise renting.

In [17]:
# Changing the datatype of date column
bike_df['date']=pd.to_datetime(bike_df['date'], format = "%d/%m/%Y")
# Creating a column for days
bike_df['days'] = bike_df['date'].dt.day_name()

In [18]:
# count of average rentals for each hour
bike_df.groupby('hour')['rented_count'].mean()

hour
0      541.460274
1      426.183562
2      301.630137
3      203.331507
4      132.591781
5      139.082192
6      287.564384
7      606.005479
8     1015.701370
9      645.983562
10     527.821918
11     600.852055
12     699.441096
13     733.246575
14     758.824658
15     829.186301
16     930.621918
17    1138.509589
18    1502.926027
19    1195.147945
20    1068.964384
21    1031.449315
22     922.797260
23     671.126027
Name: rented_count, dtype: float64

The 'hour' column in the dataset has 24 values from 0 to 23 hours so we considerd grouping them in some periods for better analysis and decided to divide the 24 hours in 3 periods of 8 hours each. The 3 periods are defined as:
* 'night' : starts from mid-night at 0 hours and lasts till 7 hours in the morning.
* 'day' : starts in the morning at 8 hours and lasts till 15 hours in the after-noon.
* 'evening' : starts in the after-noon at 16 hours and lasts till 23 hours in the night.


In [23]:
# Forming the 'period' column with periods defined above.
bike_df['period']= bike_df['hour'].apply(lambda x: 'night' if x<8 else ( 'day' if x<16 else'evening'))

In [24]:
# Make a copy of dataframe
df=bike_df.copy()

In [25]:
# Now we drop the date column
df.drop(columns=['date'], inplace=True)

In [26]:
df.head()

,rented_count,hour,temp,humidity,wind_speed,visibility,dew_point_temp,solar_radiation,rainfall,snowfall,seasons,holiday,funct_day,days,period
0,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday,night
1,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday,night
2,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday,night
3,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday,night
4,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,Friday,night


### What all manipulations are done and insights we found?

All the manipulations are done to get a workable and analysis-friendly dataset. First we renamed the columns to sensible names.

The 'date' column in the dataset we are provided with was of object type so to make it usable we changed it to 'datetime' format. The analysis of day-wise renting would make more sense so we got a column for days from date column. We'll see various relationships in the next section.


In order to get some insight from various day-segments we divide the 24 hours of day in 3 parts of 8 hours. It will make us aware of some renting pattern in the next section.